In [19]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [20]:
import datetime
import pathlib
import math

from dateutil.relativedelta import relativedelta

In [21]:
year = datetime.date.today().year
month = datetime.date.today().month
today = datetime.date.today()
pre_month = today - relativedelta(months=1)
pre_month = datetime.date.strftime(pre_month, '%m')

In [22]:
print(pre_month)

10


In [24]:
gsheet_url = "https://docs.google.com/spreadsheets/d/1nILAyd6qEonycBJ0sBFUKcpoGutabed6gAlFdEnT1mQ/edit#gid=0"

smn_uidベースのアプリリクエスト

In [25]:
query = (
    f"""
    with tmp as (
    select 
    concat_ws('-',year,month) as yymm
    ,case 
    when cast(is_app as int) = 1 and user_agent_os_family in('Android') then 'Androidアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('Android') then 'Androidウェブ'
    when cast(is_app as int) = 1 and user_agent_os_family in('iOS') then 'iOSアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('iOS') then 'iOSウェブ'
    else 'PC'
    end as OS
    ,sum(req) req
    ,sum(0) res
    ,sum(smn_uid_count) sync
    from dm.domain_req 
    where 
    year = '{year}' 
    and month = "{pre_month}" 
    and ssp_id > 0 
    group by 1,2
    
    union all
    
    select 
    concat_ws('-',year,month) as yymm
    ,case 
    when cast(is_app as int) = 1 and user_agent_os_family in('Android') then 'Androidアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('Android') then 'Androidウェブ'
    when cast(is_app as int) = 1 and user_agent_os_family in('iOS') then 'iOSアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('iOS') then 'iOSウェブ'
    else 'PC'
    end as OS
    ,sum(0) as req
    ,sum(case when tactics_id > 0 then res else 0 end) res
    ,sum(0) as sync
    from dm.domain_res
    where 
    year = '{year}' 
    and month = "{pre_month}" 
    and ssp_id > 0
    and tactics_id > 0 
    and dsp_company_id=1
    group by 1,2
    )
    select 
    yymm
    ,OS
    ,sum(req) as req
    ,sum(res) as res
    ,sum(sync) as sync_req
    from tmp
    group by 1,2

    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_req = ir.sql_to_pandas(query)
    gs = GSheet()
    #gs.add_data(url, 0, df_req, [0, 0], True)
    
    #gs = GSheet()
    sheet_index = 5
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    w

In [26]:
df_req

,yymm,os,req,res,sync_req
0,2019-10,Androidウェブ,207815920129,150182009441,196275450070
1,2019-10,iOSアプリ,47933624455,11119733510,16393345378
2,2019-10,iOSウェブ,62436453613,29730781272,37856862402
3,2019-10,PC,155926878509,85713701663,141854971224
4,2019-10,Androidアプリ,35668799451,12711226430,17382961582


売上<br>
ad_type=0,4 がRTA売上（DYC含む）<br>
ad_type=4がDYC売上

In [27]:
query = (
    f"""
    select 
    concat_ws('-',year,month) as yymm
    ,case 
    when cast(is_app as int) = 1 and user_agent_os_family in('Android') then 'Androidアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('Android') then 'Androidウェブ'
    when cast(is_app as int) = 1 and user_agent_os_family in('iOS') then 'iOSアプリ'
    when cast(is_app as int) = 0 and user_agent_os_family in('iOS') then 'iOSウェブ'
    else 'PC'
    end as OS
    ,sum(imp)+0.0 imp
    ,sum((1+csm.margin/100)*win_price)/1000000 sales
    ,sum(case when ad_type in (0,4) then (1+csm.margin/100)*win_price/1000000 else 0 end ) as rtg_sales_incdyc
    ,sum(case when ad_type in (4) then (1+csm.margin/100)*win_price/1000000 else 0 end ) as dyc_sales 
    from dm.domain_ads ads 
    inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
    inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
    inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
    inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
    where 
    ads.year = '{year}'
    and ads.month = '{pre_month}' 
    and imp > 0 
    and ads.dsp_company_id=1
    group by 1,2
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_sales = ir.sql_to_pandas(query)
    df_sales = df_sales.astype({'sales':float,'rtg_sales_incdyc':float,'dyc_sales':float})
    #gs = GSheet()
    #gs.add_data(url, 1, df_sales, [0, 0], True)
    
    gs = GSheet()
    sheet_index = 6
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_sales, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

req(シンク済み）<br>
シンク済みreqは、下記SQLでデータ抽出し、PCはall_syncからSPを引いたものとする<br>
★台湾に配信する日本案件の可能性もあるので、ここでのreqとしては、台湾除外はしていない

query = (
    f"""
    select
    concat_ws('-',year,month) as yymm
    ,case 
    when cast(is_app as int) = 1 then 'Androidアプリ' else 'Androidウエブ'
    end as type
    ,sum(smn_uid_count)+0.0 sync
    from dm.domain_req 
    where 
    user_agent_os_family = 'Android' 
    and year = "{year}" 
    and month ="{pre_month}"   
    group by 1,2
    order by 1,2
    
    union all
    select
    concat_ws('-',year,month) as yymm
    ,case 
    when cast(is_app as int) = 1 then 'iOSアプリ' else 'iOSウエブ'
    end as type
    ,sum(smn_uid_count)+0.0 sync
    from dm.domain_req
    where 
    user_agent_os_family = 'iOS' 
    and year ="{year}" 
    and month ="{pre_month}"
    group by 1,2
    order by 1,2
    
    union all
    select
    concat_ws('-',year,month) as yymm
    ,'ALL' as type
    ,sum(smn_uid_count)+0.0 sync
    from dm.domain_req 
    where 
    year ="{year}" 
    and month ="{pre_month}"
    group by 1,2
    order by 1,2
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_sync = ir.sql_to_pandas(query)
    gs = GSheet()
    #gs.add_data(url, 3, df_sync, [0, 0], True)
    #gs = GSheet()
    sheet_index = 2
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_sync, col_offset)

R数<br>

In [28]:
query = (
    f"""
    select
    concat_ws('-',year,month) as yymm
    ,'Android' as type
    ,count(distinct smn_uid) as r_uu
    ,count(smn_uid) as r
    from dsplog.adserver 
    where
    action_type = 'R' 
    and user_agent_os_family = 'Android' 
    and year = "{year}" 
    and month ="{pre_month}"
    and dsp_company_id = 1
    group by 1,2
    
    union all
    select
    concat_ws('-',year,month) as yymm
    ,'iOS' as type
    ,count(distinct smn_uid) as r_uu
    ,count(smn_uid) as r
    from dsplog.adserver 
    where
    action_type = 'R' 
    and user_agent_os_family = 'iOS' 
    and year ="{year}" 
    and month ="{pre_month}"
    and dsp_company_id = 1
    group by 1,2
    
    union all
    select
    concat_ws('-',year,month) as yymm
    ,'PC' as type
    ,count(distinct smn_uid) as r_uu
    ,count(smn_uid) as r
    from dsplog.adserver 
    where
    action_type = 'R' 
    and user_agent_os_family not in ('iOS', 'Android')
    and year ="{year}" 
    and month ="{pre_month}"
    and dsp_company_id = 1
    group by 1,2
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_r = ir.sql_to_pandas(query)
    gs = GSheet()
    #gs.add_data(url, 4, df_r, [0, 0], True)
    #gs = GSheet()
    sheet_index = 7
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_r, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

ios os別<br>
ios リターゲ os別　UUと件数は、下記SQLでデータ抽出する

In [29]:
query = (
    f"""
    select
    concat_ws('-',year,month) as yymm
    ,user_agent_os_major
    ,count(smn_uid) as count
    ,count(distinct smn_uid) as uu
    from 
    dsplog.adserver
    where 
    user_agent_os_family in ('iOS') 
    and action_type = "R" 
    and year = '{year}' 
    and month = '{pre_month}'
    and user_agent_os_major is not null
    group by yymm,user_agent_os_major
    order by yymm,user_agent_os_major
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_os = ir.sql_to_pandas(query)
    gs = GSheet()
    #gs.add_data(url, 5, df_os, [0, 0], True)
    #gs = GSheet()
    sheet_index = 8
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_os, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

アプリ上位ドメイン<br>
appstore_categoriesがカンマ区切りで最大4つでてくるので、エクセルで分けてください。<br>
アプリカテゴリのシートでIDをvlookupで紐づけて完成です。<br>
※アプリカテゴリは頻繁に更新はないですが、最新版はagency_console.appstore_ctgrから抽出できます。<br>
エクセルは、Logicadポータルにアップロードする。

In [30]:
query = (
    f"""
    select title
    ,a.domain
    ,sum(smn_uid_count)+0.0 req_sync_all
    ,sum(case when native_request=true and concat_ws('x', CAST(width AS string), CAST(height AS string)) !='0x0' then smn_uid_count else 0 end)+0.0 'ネイティブ静止画兼用_req'
    ,sum(case when native_request=true and concat_ws('x', CAST(width AS string), CAST(height AS string)) ='0x0' then smn_uid_count else 0 end)+0.0 'ネイティブ専用_req'
    ,sum(case when native_request!=true then smn_uid_count else 0 end)+0.0 '静止画専用_req' 
    ,appstore_categories
    from dm.domain_req a 
    inner join agency_console.agency_domain_master b using(domain_id)
    where 
    year = '{year}'
    and month = '{pre_month}'
    and user_agent_os_family in ('iOS','Android') 
    and is_app =true 
    and child_ssp_id not in(18,22,29) 
    group by 1,2,7 
    having req_sync_all>0 
    order by 3 desc 
    limit 3000

    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_app = ir.sql_to_pandas(query)
    #gs = GSheet()
    #gs.add_data(url, 6, df_app, [0, 0], True)
    #gs = GSheet()
    #sheet_index = 0
    #col_offset=0
    #gs.append_data(url, sheet_index, df, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [31]:
df_app.head()

,title,domain,req_sync_all,ネイティブ静止画兼用_req,ネイティブ専用_req,静止画専用_req,appstore_categories
0,地元の掲示板「ジモティー」オークションより簡単・無料,jp.jmty.app2.googleplaypackagename.example.com,1.744082e+09,0.0,1.041182e+09,7.029008e+08,60020
1,ソリティアデイリー,1456989345.itunesid.example.com,1.261282e+09,0.0,7.090456e+06,1.254192e+09,"60506,60526,60530"
2,Ameba（アメーバ）,349442137.itunesid.example.com,1.013481e+09,0.0,7.298993e+08,2.835816e+08,60517
3,2chMate,jp.co.airfront.android.a2chMate.googleplaypack...,1.011818e+09,38077279.0,4.397000e+04,9.736968e+08,60021
4,Solitaire Daily - Card Games,com.bigcake.android.solitairedaily.googleplayp...,8.621448e+08,0.0,2.930000e+03,8.621419e+08,60030


In [32]:
splitted=df_app['appstore_categories'].str.split(",",expand=True)
splitted.columns = ['id_1','id_2','id_3','id_4']
del df_app['appstore_categories']

In [33]:
splitted.head()

,id_1,id_2,id_3,id_4
0,60020,None,None,None
1,60506,60526,60530,None
2,60517,None,None,None
3,60021,None,None,None
4,60030,None,None,None


In [34]:
df_tmp = pd.concat([df_app,splitted], axis=1)

In [35]:
df_tmp.head()

,title,domain,req_sync_all,ネイティブ静止画兼用_req,ネイティブ専用_req,静止画専用_req,id_1,id_2,id_3,id_4
0,地元の掲示板「ジモティー」オークションより簡単・無料,jp.jmty.app2.googleplaypackagename.example.com,1.744082e+09,0.0,1.041182e+09,7.029008e+08,60020,None,None,None
1,ソリティアデイリー,1456989345.itunesid.example.com,1.261282e+09,0.0,7.090456e+06,1.254192e+09,60506,60526,60530,None
2,Ameba（アメーバ）,349442137.itunesid.example.com,1.013481e+09,0.0,7.298993e+08,2.835816e+08,60517,None,None,None
3,2chMate,jp.co.airfront.android.a2chMate.googleplaypack...,1.011818e+09,38077279.0,4.397000e+04,9.736968e+08,60021,None,None,None
4,Solitaire Daily - Card Games,com.bigcake.android.solitairedaily.googleplayp...,8.621448e+08,0.0,2.930000e+03,8.621419e+08,60030,None,None,None


In [36]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    gs = GSheet()
    sheet_index = 10
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_tmp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.utils.gsheet:loa